# Laboratorio de Introducción al Procesamiento de Lenguaje Natural (2024)

**Grupo**: PG01

**Integrantes**: Emiliano Viotti

---


En este laboratorio vamos a poner en práctica algunos de los conceptos vistos en el curso, como la *mínima distancia de edición* para medir la cercanía entre dos tiras, los *embeddings* como representación semántica de los textos, los *modelos de n-gramas* para generar texto y un problema de clasificación de tres clases.

Los ejercicios que tienen que resolver están señalizados por números del 0️⃣al 7️⃣. Los resultados y conclusiones que surjan de esos ejercicios los van a comunicar en un informe (con un **máximo de 6 páginas**). En la sección "¿Qué se espera del informe?" de la letra del laboratorio van a encontrar una guía para escribirlo.



---

Antes que nada, vamos a instalar e importar NLTK y descargar ["Cuentos de Amor de Locura y de Muerte"](https://es.wikipedia.org/wiki/Cuentos_de_amor_de_locura_y_de_muerte) de Horacio Quiroga del [repositorio Gutemberg](https://www.gutenberg.org/ebooks/13507). Esto ya está implementado, y el libro va a quedar guardado en la variable `quiroga_raw`.

In [67]:
%%capture
import re
import nltk
import pandas as pd
nltk.download('punkt')

!wget https://www.gutenberg.org/cache/epub/13507/pg13507.txt
with open("pg13507.txt","r") as f:
    quiroga_raw = f.read()


## 0️⃣Limpieza básica del corpus

Si examinamos el string cargado en `quiroga_raw`, vemos que al inicio y al final hay texto en inglés, además de que algunos saltos de línea ocurren a mitad de las oraciones.

Como primer paso, diseñe expesiones regulares que limpien `quiroga_raw` para que:


* quede solamente el texto que está entre "#Cuentos de Amor de Locura y de Muerte#" y "FIN\n"
* saquen aquellos saltos de línea que tengan a su derecha una palabra que comience con letra minúscula  

El texto limpio resultante deberá quedar guardado en la variable `quiroga`. Más adelante van a poder probar con otros preprocesamientos complementarios a esta limpieza básica.



In [68]:
#En esta celda les dejamos esta asignación para que el notebook sea funcional.
#Implementen en esta celda la limpieza correspondiente.

# 1. Extracción del texto
print("Extrayendo texto...")
quiroga_text_extract_pattern = r"#Cuentos de Amor de Locura y de Muerte#(.*)FIN\n"  # Expresion regular
match = re.search(quiroga_text_extract_pattern, quiroga_raw, re.DOTALL)
if match:
    quiroga_text = match.group(1).strip()  # Remuevo espacios en blanco
    print(f"Texto original: {len(quiroga_raw)} caracteres. \nTexto extraído: {len(quiroga_text)} caracteres.")
else:
    print("Algo anda mal!")

# 2. Limpieza saltos de línea
print("\n\nLimpiando saltos de línea en sentencias...")
breakline_lowercase_pattern = r"\n([a-z])"
quiroga = re.sub(breakline_lowercase_pattern, r" \1", quiroga_text)  # Reemplazo \n por un ' '
print(f"Texto original: {len(quiroga_text)} caracteres. \nTexto extraído: {len(quiroga)} caracteres.")

print("\n\n")
print("Texto Limpio\n")
print(quiroga[:1000])

Extrayendo texto...
Texto original: 312980 caracteres. 
Texto extraído: 293434 caracteres.


Limpiando saltos de línea en sentencias...
Texto original: 293434 caracteres. 
Texto extraído: 293434 caracteres.



Texto Limpio

HORACIO QUIROGA

1917




#INDICE#


Una estación de amor
Los ojos sombríos
El solitario
La muerte de Isolda
El infierno artificial
La gallina degollada
Los buques suicidantes
El almohadón de pluma
El perro rabioso
A la deriva
La insolación
El alambre de púa
Los Mensú
Yaguaí
Los pescadores de vigas
La miel silvestre
Nuestro primer cigarro
La meningitis y su sombra









#UNA ESTACION DE AMOR#




#Primavera#


Era el martes de carnaval. Nébel acababa de entrar en el corso, ya al oscurecer, y mientras deshacía un paquete de serpentinas, miró al carruaje de delante. Extrañado de una cara que no había visto la tarde anterior, preguntó a sus compañeros:


--¿Quién es? No parece fea.

--¡Un demonio! Es lindísima. Creo que sobrina, o cosa así, del doctor
Arrizabalaga. 

## Primera parte: Modelos de lenguaje de n-gramas

Ahora que tenemos el texto más limpio, vamos a entrenar modelos de lenguaje de n-gramas sobre él. Para eso, les vamos a dar definidas e implementadas las funciones:
- `train_language_model`, que dado un natural `n` y un `corpus`, entrena un modelo de lenguaje basado en n-gramas. El parámetro `n` tiene el valor 4 por defecto.
- `language_model_inference`, que dado un modelo de lenguaje ya entrenado y una `prompt`, da como salida tantos tokens sucesivos como indique la variable `length`. Este largo tiene por defecto el valor 25, y la semilla `seed` tiene por defecto el valor 2024.

In [69]:
def train_language_model (corpus, n = 4):
  padded_tokens = [list(nltk.lm.preprocessing.pad_both_ends(nltk.tokenize.word_tokenize(corpus, language='spanish'), n=n))]

  train, vocab = nltk.lm.preprocessing.padded_everygram_pipeline(n, padded_tokens)

  language_model = nltk.lm.MLE(n)
  language_model.fit(train,vocab)

  return language_model

def language_model_inference (language_model, prompt, length = 25, seed = 2024):
  tokens = language_model.generate(length, text_seed=nltk.tokenize.word_tokenize(prompt), random_seed=seed)
  return tokens


### 1️⃣Entrenamiento e inferencia

Ahora que ya están definidas las funciones de entrenamiento e inferencia de los modelos de lenguaje, entrenen diferentes modelos variando:
- el parámetro `n`, por ejemplo con `n=3`, `n=2` y el valor por defecto, `n=4`
- el parámetro `corpus`, que se consigue al probar con diferentes pipelines de preprocesamieto. Pueden intentar pasar todo a minúsculas, sustituir los saltos de línea por espacios en blanco y todo lo que tengan ganas de probar. También pueden incluir el experimento de no preprocesar; es decir, que el pipeline de preprocesamiento sea la identidad.

Luego, con cada uno de esos modelos, generen texto pasándole como `prompt`,  "las vacas", "el cielo", y una opción más que encuentren interesante. Aunque no es necesario, pueden probar otros valores de `length` y `seed`, más allá de los que están por defecto.

💡 Les recomendamos que cada pipeline de preprocesamiento sea una función independiente. De esa manera, se facilita la reproducibilidad de los experimentos y su comparación, e.g. `n=3` preprocesando con `pipeline1()`, `n=3` con `pipeline2()`, `n=2` con `pipeline1()` y `n=2` con `pipeline2()`.



### Pre-procesamientos a probar

Se propone experimentar con los siguientes pipelines de datos, que combinan diferentes pre procesamientos y parámetros en la etapa de vectorización del texto.

| Nombre      | Parámetros                                      | Objetivo                                           |
|-------------|-------------------------------------------------|----------------------------------------------------|
| pipeline_1  | n=4                                             | Probar el efecto de n grande                       |
| pipeline_2  | n=3                                             | Probar el efecto reducir n                         |
| pipeline_3  | n=2                                             | Probar el efecto reducir n                         |
| pipeline_4  | n=5                                             | Probar el efecto n más grande                      |
| pipeline_5  | n=4, lowercase()                                | Para un n fijo, agregar convertir a minúsculas                     |
| pipeline_6  | n=4, lowercase(), replace(“\n”, “ ”)            | Para un n fijo, agregar eliminar todos los saltos                  |
| pipeline_7  | n=4, lowercase(), replace(“\n”, “ ”), replace(punc_symbols, “ “) | Para un n fijo, además eliminar símbolos de puntuación y caracteres especiales |


In [70]:
#Completar con los diferentes experimentos, llamando a las funciones train_language_model y language_model_inference

"""
Experimentos a realizar:

1. Variar n en [4, 3, 2]
2. Jugar con pre-procesamientos.

Pre-procesamientos:
1. Nada
2. lowercase
3. replace(\n, " ")
"""

def pipeline_1():
  """
  Este pipeline entrena modelo con n=4 y sin pre-procesamiento al corpus (más allá del inicial).
  Esto nos permite observar la capacidad del modelo cuando se toman n-gramas de a cuatro palabras, lo cual por el
  tamaño del corpus, es probable que genere sobre ajustes.
  """
  print(f"pipeline_1 -- Parámetros: n={4}, pre-process=None")
  language_model = train_language_model(corpus=quiroga, n=4)
  return language_model

def pipeline_2():
  """
  Este pipeline entrena modelo con n=3 y sin pre-procesamiento al corpus (más allá del inicial).
  Esto nos permite observar como el modelo gana generalidad al no sobre ajustarse tanto pero seguramente
  pierde capacidad de generación al tomar contexto de palabras reducido.
  """
  print(f"pipeline_2 -- Parámetros: n={3}, pre-process=None")
  language_model = train_language_model(corpus=quiroga, n=3)
  return language_model

def pipeline_3():
  """
  Este pipeline entrena modelo con n=2 y sin pre-procesamiento al corpus (más allá del inicial).
  Similar al anterior.
  """
  print(f"pipeline_3 -- Parámetros: n={2}, pre-process=None")
  language_model = train_language_model(corpus=quiroga, n=2)
  return language_model

def pipeline_4():
  """
  Este pipeline entrena modelo con n=5 y sin pre-procesamiento al corpus (más allá del inicial).
  Esto nos permite terminar de afianzar la idea de que se está sobre ajustando el modelo al texto provisto.
  """
  print(f"pipeline_4 -- Parámetros: n={5}, pre-process=None")
  language_model = train_language_model(corpus=quiroga, n=5)
  return language_model

def pipeline_5():
  """
  Este pipeline entrena modelo con n=4 y lowercase al corpus
  """
  print(f"pipeline_5 -- Parámetros: n={4}, pre-process=lowercase()")

  # Pre-procesamiento
  pre_process_corpus = quiroga.lower()

  # Entrenamiento
  language_model = train_language_model(corpus=pre_process_corpus, n=4)
  return language_model

def pipeline_6():
  """
  Este pipeline entrena modelo combinando: n=4, lowercase y replace(\n, " ").

  Al ser un texto pequeño, pasar a minúsculas y eliminar saltos de línea nos permite manejar
  varias palabras como la misma, mejorando la capacidad de comprensión de un modelo simple como n-gramas.
  """
  print(f"pipeline_6 -- Parámetros: n={4}, pre-process=[lowercase(), replace(\n, ' ')]")

  # Pre-procesamiento
  pre_process_corpus = quiroga.replace("\n", " ")

  # Entrenamiento
  language_model = train_language_model(corpus=pre_process_corpus, n=4)
  return language_model

def pipeline_7():
  """
  Este pipeline entrena modelo combinando: n=4, lowercase y replace(\n, " ").
  """
  print(f"pipeline_7 -- Parámetros: n={4}, pre-process=[lowercase(), replace([\n, pubc_symbols], ' ')]")

  # Pre-procesamiento
  pre_process_corpus = quiroga.replace("\n", " ")  # Elimino saltos de línea
  pre_process_corpus = re.sub(r"[^\w\s]", " ", pre_process_corpus)  # Elimino símbolos de puntuación
  pre_process_corpus = re.sub(r"\s+", " ", pre_process_corpus).strip()  # Elimino espacios en blanco repetidos

  # Entrenamiento
  language_model = train_language_model(corpus=pre_process_corpus, n=4)
  return language_model


# Generamos los language models
models = [pipeline_1(), pipeline_2(), pipeline_3(), pipeline_4(), pipeline_5(), pipeline_6(), pipeline_7()]
#models = [pipeline_5()]

# Generamos textos
results = []
texts = ["las vacas", "el cielo", "Esteban Podeley"]
#texts = ["las vacas"]

for text in texts:

  print(f"\nGenerando para text='{text}'")
  text_results = []
  for i, model in enumerate(models):
    output_tokens = language_model_inference(language_model=model, prompt=text)
    model_name = f"pipeline_{i+1}"
    print(f"model='{model_name}' output={output_tokens}")
    results.append({"pipeline": model_name, "prompt": text, "output": " ".join(output_tokens)})

pipeline_1 -- Parámetros: n=4, pre-process=None
pipeline_2 -- Parámetros: n=3, pre-process=None
pipeline_3 -- Parámetros: n=2, pre-process=None
pipeline_4 -- Parámetros: n=5, pre-process=None
pipeline_5 -- Parámetros: n=4, pre-process=lowercase()
pipeline_6 -- Parámetros: n=4, pre-process=[lowercase(), replace(
, ' ')]
pipeline_7 -- Parámetros: n=4, pre-process=[lowercase(), replace([
, pubc_symbols], ' ')]

Generando para text='las vacas'
model='pipeline_1' output=['dormitaban', 'al', 'sol', 'ya', 'caliente', ',', 'rumiando', '.', 'Pero', 'cuando', 'está', 'conmigo', ',', 'entonces', 'no', 'aparta', 'los', 'ojos', 'de', 'mi', 'mujer', 'y', 'yo', ',', 'con']
model='pipeline_2' output=['dormitaban', 'al', 'sol', 'ya', 'caliente', ',', 'rumiando', '.', 'Pero', 'en', 'el', 'patio', 'y', 'Alfonso', 'la', 'llamó', 'en', 'silencio', '.', 'Pasábanse', 'horas', 'sin', 'oir', 'el', 'angustioso']
model='pipeline_3' output=['dormitaban', 'al', 'fin', 'se', 'había', 'reforzado', 'su', 'corazón', '

In [71]:
# Muestro los resultados en un dataframe para visualizarlos en formato tabla
df_1 = pd.DataFrame(results)
df_1

,pipeline,prompt,output
0,pipeline_1,las vacas,"dormitaban al sol ya caliente , rumiando . Per..."
1,pipeline_2,las vacas,"dormitaban al sol ya caliente , rumiando . Per..."
2,pipeline_3,las vacas,dormitaban al fin se había reforzado su corazó...
3,pipeline_4,las vacas,"dormitaban al sol ya caliente , rumiando . Per..."
4,pipeline_5,las vacas,"estaban inmóviles , mirando fijamente el verde..."
5,pipeline_6,las vacas,"dormitaban al sol ya caliente , rumiando . Per..."
6,pipeline_7,las vacas,dormitaban al sol ya caliente rumiando Pero cu...
7,pipeline_1,el cielo,"constantemente encapotado y lluvioso , provocá..."
8,pipeline_2,el cielo,"constantemente encapotado y lluvioso , provocá..."
9,pipeline_3,el cielo,de que está todo el piso . Alrededor del mengu...


**OBSERVACIONES**

* Algunos pipelines producen sentencias con "sentído" y gramaticalmente correctas (o cerca de serlas). Este es el caso de los pipelines: pipeline_1, pipeline_2, pipeline_4 al pipeline_7.

* Algunos pipelines producen sentencias con menor "sentido" y gramaticalmente incorrectas. Por ejemplo los pipelines: pipeline_2 y pipeline_3. Se puede observar así que el efecto de reducir el N en los n-gramas tiene un efecto negativo en la generación de sentencias gramaticalmente correctas.

* De lo anterior se puede deducir que tomar un N más grande (N=4 o N=5) para los n-gramas tiene un efecto positivo en la generación de sentencias gramaticalmente correctas.

* Pasar a minúsculas no parece tener un impacto negativo en la generación (pipeline_5). Por el contrario, se genera una sentencia bastante diferente al resto de los experimentos. Esto puede deberse a que al pasar a lowercase palabras como "dormitaban" y "Dormitaban" pasan a ser la misma, aumentando la cantidad de ejemplos en el corpus para esa palabra.

* Remover los saltos de línea adicionales no parece producir un impacto positivo o negativo por si solo.

* Remover los símbolos de puntuación no parece tener un impacto positivo. Lo único que cambia es que generamos sentencias más largas (hay más tokens disponibles) pero podemos lograr lo mismo aumentando el parámetro `length` en el método `language_model_inference`.

### 2️⃣ Mínima distancia de edición

Los modelos de lenguaje que entrenaron en la parte anterior aprendieron la distribución estadística a través de secuencias de palabras en el libro de Quiroga. Sin embargo, las inferencias que consiguieron ¿estarán idénticas en el corpus de entrenamiento?

Usando la función `edit_distance` de NLTK encuentren, para cada tira generada en la parte anterior, las 3 oraciones del libro original que tienen la menor distancia de edición asociada (es decir, el top 3). Las oraciones del libro están guardadas en la variable `quiroga_sentences`.

In [72]:
quiroga_sentences = nltk.sent_tokenize(quiroga, language='spanish')

In [73]:
print(f'La distancia entre "arbolada" y "escapada" es {nltk.edit_distance("arbolada", "escapada")}, pero entre "arbolada" y "arbolado" es {nltk.edit_distance("arbolada", "arbolado")}.')

La distancia entre "arbolada" y "escapada" es 5, pero entre "arbolada" y "arbolado" es 1.


In [74]:
import numpy as np

def get_knn(text: str, k: int = 3, sentences: list[str] = []) -> list[str]:
  # Obtengo distancias
  distances = [nltk.edit_distance(text, sentence) for sentence in sentences]
  # Obtengo indices de los K mas cercanos
  closest_indices = np.argsort(distances)[:k]
  closest_sentences = [sentences[i] for i in closest_indices]

  return closest_sentences

# Para guardar los resultados
predictions = []

for text in texts:

  print(f"\n\nGenerando para text='{text}'")
  for i, model in enumerate(models):
    # Genero texto
    model_name = f"pipeline_{i+1}"
    output_tokens = language_model_inference(language_model=model, prompt=text)
    output_text = " ".join(output_tokens)

    print(f"model='{model_name}' output='{output_text}'")

    # Obtengo top_k vectores mas cercanos
    nearest_k = get_knn(text=output_text, k=3, sentences=quiroga_sentences)
    print(f"neighbors={nearest_k}")

    for idx, neighbor_vec in enumerate(nearest_k):
      # Guardo resultados
      predictions.append({
                  "prompt": text,
                  "model_name": model_name,
                  "output_text": output_text,
                  "idx": idx+1,
                  "neighborh": neighbor_vec
              })




Generando para text='las vacas'
model='pipeline_1' output='dormitaban al sol ya caliente , rumiando . Pero cuando está conmigo , entonces no aparta los ojos de mi mujer y yo , con'
neighbors=['Pero cuando está conmigo, entonces no aparta los ojos de ellos.', 'Arrizabalaga y la señora se reían, volviéndose a menudo, y la joven no apartaba casi sus ojos de Nébel.', 'El caballo, por mayor intimidad de trato, es sensiblemente más afecto al hombre que la vaca.']
model='pipeline_2' output='dormitaban al sol ya caliente , rumiando . Pero en el patio y Alfonso la llamó en silencio . Pasábanse horas sin oir el angustioso'
neighbors=['Celia, mi tía mayor, que había concluído de dormir la siesta, cruzó el patio y Alfonso la llamó en silencio con la mano.', 'Al bajar el sol volvieron, pero Berta quiso saludar un momento a sus vecinas de enfrente.', 'Volvió a su cobertizo, y en el camino sintió un ligero cosquilleo en la espalda.']
model='pipeline_3' output='dormitaban al fin se había reforzado s

In [75]:
# Creo DataFrame
df_2 = pd.DataFrame(predictions, columns=["prompt", "model_name", "output_text", "idx", "neighborh"])
df_2

,prompt,model_name,output_text,idx,neighborh
0,las vacas,pipeline_1,"dormitaban al sol ya caliente , rumiando . Per...",1,"Pero cuando está conmigo, entonces no aparta l..."
1,las vacas,pipeline_1,"dormitaban al sol ya caliente , rumiando . Per...",2,"Arrizabalaga y la señora se reían, volviéndose..."
2,las vacas,pipeline_1,"dormitaban al sol ya caliente , rumiando . Per...",3,"El caballo, por mayor intimidad de trato, es s..."
3,las vacas,pipeline_2,"dormitaban al sol ya caliente , rumiando . Per...",1,"Celia, mi tía mayor, que había concluído de do..."
4,las vacas,pipeline_2,"dormitaban al sol ya caliente , rumiando . Per...",2,"Al bajar el sol volvieron, pero Berta quiso sa..."
...,...,...,...,...,...
58,Esteban Podeley,pipeline_6,", peones de obraje , volvían a Posadas en el _...",2,Reverberaba ahora delante de ellos un pequeño ...
59,Esteban Podeley,pipeline_6,", peones de obraje , volvían a Posadas en el _...",3,"Me desperté, y volví a soñar: el tal salón de ..."
60,Esteban Podeley,pipeline_7,peones de obraje volvían a Posadas en el _Sile...,1,"Sobre la honda ligadura del pañuelo, la carne ..."
61,Esteban Podeley,pipeline_7,peones de obraje volvían a Posadas en el _Sile...,2,"Me desperté, y volví a soñar: el tal salón de ..."


**OBSERVACIONES**

- Observando el top 3 de oraciones más cercanas (en base a la distancia de edición), se puede ver que para algunos preprocesamientos el top 3 de oraciones incluye una oración bastante similar a la generada por el modelo, mientras que en otros casos no.

- En particular para los pipelines pipeline_2 y pipeline_3 el top 3 de sentencias NO incluye una sentencia que a simple vista sea similar al texto generado. Estos pipelines tienen en común que toman un N pequeño para la vectorización en n-gramas (N=3, N=2).

- Los siguientes pipelines, producen oraciones que a simple vista coinciden en gran medida con alguna de las oraciones en el top 3: pipeline_1, pipeline_4, pipeline_5, pipeline_6 y pipeline_7. Estos pipelines tienen en común el uso de un N moderado para la generación de n-gramas (N=4, N=5).

- Pasar a minúsculas, remover símbolos de puntuación y saltos de línea, no parece afectar negativamente las distancias. En particular se aprecia al menos una oración dentro del top 3 muy similar o idéntica al texto generado, cuando se utilizan estos preprocesamientos.

Por otro lado, es interesante entender en el contexto del texto original, a que fragmento se puede estar sobre ajustando el texto generado. En particular seleccionando el texto generado con el **pipeline_6**, vemos que el inicio de la oración se encuentra textual en el siguiente pasaje:

> Detrás de él, **las vacas**
> dormitaban al sol ya caliente, rumiando.
>
> Pero cuando los pobres caballos pasaron por el camino, ellas abrieron
>los ojos despreciativas:

Basado en estas observaciones y en las observaciones de la parte anterior, parecería que el **pipeline_6** puede ser un muy buen pipeline a utilizar para el resto de los experimentos (pipeline "ganador"). No obstante, notar que estamos optando por un pipeline, por su capacidad de sobre ajustar al texto generado (esto podría no ser la mejor métrica de evaluación).

### 3️⃣ Modelo neuronal para capturar la semántica

Ahora repitan el mismo procedimiento de la parte anterior pero usando el [modelo neuronal E5](https://huggingface.co/intfloat/multilingual-e5-large), basado en la arquitectura BERT, que intenta representar el significado de un texto en un vector de largo fijo.

Para hallar los tres vectores más cercanos a un vector dado, utilicen el algoritmo [Nearest Neighbors implementado en Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html).

💡Van a tener que codificar (con vectores) tanto lo generado por el modelo de n-gramas como todas las oraciones guardadas en `quiroga_sentences`.

💡Asegúrense de estar usando una GPU en el entorno de ejecución de Google Colab, para poder acelerar el procesamiento con E5.

In [76]:
#Completar con la instalación de las bibliotecas necesarias,
#el código que halla la representación vectorial de los textos
#y el algoritmo para hallar el top 3 con Nearest Neighbors.

import torch
from torch.utils.data import DataLoader
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
  last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
  return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def generate_vector_from_text(batch, model, tokenizer, device):
  """
  Inspirado por https://discuss.huggingface.co/t/is-transformers-using-gpu-by-default/8500/2
  en como manejar en GPU los vectrores resolviendo algunos problemas que tuve con Colab.
  """

  # Tokenizo el batch
  batch_dict = tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)

  with torch.no_grad():  # Deshabilito gradient calculation
      outputs = model(**batch_dict)
      embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
      embeddings_cpu = embeddings.cpu()  # Muevo embeddings a CPU para liberar memoria GPU

  # Limpio memoria
  del outputs, embeddings, batch_dict
  torch.cuda.empty_cache() if torch.cuda.is_available() else None

  return embeddings_cpu

def generate_vectors_from_texts(texts, model, tokenizer, device):
  texts_embeddings = []

  # Defino data Loader para cargar eficientemente los vectores
  batch_size = 16  # Adjust the batch size based on your memory availability
  data_loader = DataLoader(texts, batch_size=batch_size, shuffle=False)

  for batch in data_loader:
    batch_embeddings = generate_vector_from_text(batch=batch, model=model, tokenizer=tokenizer, device=device)
    texts_embeddings.append(batch_embeddings)

  # Concatenate all embeddings after processing
  texts_embeddings = torch.cat(texts_embeddings, dim=0)
  return texts_embeddings

In [77]:
# En esta celda generamos los vectores para todas las sentencias de
# quiroga y para los textos de prueba

# Device GPU si hay
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Init tokenizer y Modelo
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')
model.to(device)

# 1. Texts
#input_texts =  ["las vacas", "el cielo", "Esteban Podeley"]
input_texts = df_1["output"].to_list()  # Uso todos los textos generados en df_1
texts_embeddings = generate_vectors_from_texts(texts=input_texts, model=model, tokenizer=tokenizer, device=device)
print(texts_embeddings.shape)

# 2. Quiroga Sentences
quiroga_embeddings = generate_vectors_from_texts(texts=quiroga_sentences, model=model, tokenizer=tokenizer, device=device)
print(quiroga_embeddings.shape)


torch.Size([21, 1024])
torch.Size([3540, 1024])


In [78]:
# En esta celda calculamos los vecinos más cercanos
from sklearn.neighbors import NearestNeighbors

def get_knn_using_sklearn(texts_embeddings, quiroga_embeddings, torch_tensor=True):

  # 3. Hallo top 3 con Nearest Neighbors.
  neigh = NearestNeighbors(n_neighbors=3, radius=0.4)
  neigh.fit(quiroga_embeddings)

  predictions = []  # Para guardar las predicciones

  # Corro knn en batch para todos los texts
  if torch_tensor:
    distances, indices = neigh.kneighbors(texts_embeddings.cpu().detach().numpy(), 3, return_distance=True)
  else:
    distances, indices = neigh.kneighbors(texts_embeddings, 3, return_distance=True)

  print(f"Distances: {distances.shape}")
  print(f"Indices: {indices.shape}")

  for i, (text_distances, text_neighboors) in enumerate(zip(distances, indices)):
    for j, (distance, n_idx) in enumerate(zip(text_distances, text_neighboors)):
      model_name = f"pipeline_{i+1}"
      neighboor = quiroga_embeddings[i]
      #predictions.append({"prompt": input_texts[i], "idx": j+1, "neighborh": quiroga_sentences[n_idx]})
      predictions.append({
              "output_text": input_texts[i],
              "idx": j+1,
              "neighbor": quiroga_sentences[n_idx]}
      )

      print(f"Text: {input_texts[i]}")
      print(f"Neigh: {quiroga_sentences[n_idx]}")

  return predictions

In [79]:
predictions = get_knn_using_sklearn(texts_embeddings, quiroga_embeddings, torch_tensor=True)

Distances: (21, 3)
Indices: (21, 3)
Text: dormitaban al sol ya caliente , rumiando . Pero cuando está conmigo , entonces no aparta los ojos de mi mujer y yo , con
Neigh: Detrás de él, las vacas dormitaban al sol ya caliente, rumiando.
Text: dormitaban al sol ya caliente , rumiando . Pero cuando está conmigo , entonces no aparta los ojos de mi mujer y yo , con
Neigh: Pero cuando está conmigo, entonces no aparta los ojos de ellos.
Text: dormitaban al sol ya caliente , rumiando . Pero cuando está conmigo , entonces no aparta los ojos de mi mujer y yo , con
Neigh: Y juro que fueron fuertes las dos horas que pasamos mi mujer y yo, con la luz prendida hasta que amaneció, ella acostada, yo sentado en la cama, vigilando sin cesar la arpillera flotante.
Text: dormitaban al sol ya caliente , rumiando . Pero en el patio y Alfonso la llamó en silencio . Pasábanse horas sin oir el angustioso
Neigh: Celia, mi tía mayor, que había concluído de dormir la siesta, cruzó el patio y Alfonso la llamó en si

In [80]:
model_names = [[f"pipeline_{i}"] * 3 for i in range(1, 8)] * 3
model_names = [item for sublist in model_names for item in sublist]
text_values = [[text] * 7*3 for text in ["las vacas", "el cielo", "Esteban Podeley"]]
text_values = text_values[0] + text_values[1] + text_values[1]

In [81]:
# Agrego valores faltantes
predictions_ext = []
for p, model_name, text in zip(predictions, model_names, text_values):
  p["model_name"] = model_name
  p["prompt"] = text
  predictions_ext.append(p)

# Creo DataFrame
df_3 = pd.DataFrame(predictions_ext, columns=["prompt", "model_name", "output_text", "idx", "neighbor"])
df_3

,prompt,model_name,output_text,idx,neighbor
0,las vacas,pipeline_1,"dormitaban al sol ya caliente , rumiando . Per...",1,"Detrás de él, las vacas dormitaban al sol ya c..."
1,las vacas,pipeline_1,"dormitaban al sol ya caliente , rumiando . Per...",2,"Pero cuando está conmigo, entonces no aparta l..."
2,las vacas,pipeline_1,"dormitaban al sol ya caliente , rumiando . Per...",3,Y juro que fueron fuertes las dos horas que pa...
3,las vacas,pipeline_2,"dormitaban al sol ya caliente , rumiando . Per...",1,"Celia, mi tía mayor, que había concluído de do..."
4,las vacas,pipeline_2,"dormitaban al sol ya caliente , rumiando . Per...",2,"Detrás de él, las vacas dormitaban al sol ya c..."
...,...,...,...,...,...
58,el cielo,pipeline_6,", peones de obraje , volvían a Posadas en el _...",2,"El _Silex_ volvió a Posadas, llevando con él a..."
59,el cielo,pipeline_6,", peones de obraje , volvían a Posadas en el _...",3,"Podeley, cuya fiebre anterior había tenido hon..."
60,el cielo,pipeline_7,peones de obraje volvían a Posadas en el _Sile...,1,"Podeley, labrador de madera, tornaba a los nue..."
61,el cielo,pipeline_7,peones de obraje volvían a Posadas en el _Sile...,2,#LOS MENSÚ#\n\n\n\n\nCayetano Maidana y Esteba...


**OBSERVACIONES**

* A diferencia de la distancia de edición, las oraciones incluidas en el top 3 están relacionadas al texto generado, fundamentalmente en el significado de la misma, en lugar de simplemente en la combinación de palabras.

* A diferencia de la distancia de edición, en algunos ejemplos más de una de las oraciones más cercanas, tiene una similitud alta con el texto generado, tanto en la semántica como en co-ocurrencia de palabras.

### Approach alternativo

En base al siguiente [intercambio](https://eva.fing.edu.uy/mod/forum/discuss.php?d=308994) en el foro del laboratorio en EVA, se deduce una forma alternativa (de más alto nivel) para utilizar los embeddings de este modelo mediante la biblioteca sentence-transformers. Por las dudas en la siguiente sección se incluye el código para realizar el mismo experimento, mediante el uso de esta biblioteca, en lugar de inferir utilizando el modelo directamente y PyTorch.

In [82]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('intfloat/multilingual-e5-large')

input_texts = df_1["output"].to_list()
texts_embeddings = model.encode(input_texts, normalize_embeddings=True)
quiroga_embeddings = model.encode(quiroga_sentences, normalize_embeddings=True)

predictions = get_knn_using_sklearn(texts_embeddings, quiroga_embeddings, torch_tensor=False)

# Creo DataFrame
df_4 = pd.DataFrame(predictions, columns=["output_text", "idx", "neighbor"])
df_4


Distances: (21, 3)
Indices: (21, 3)
Text: dormitaban al sol ya caliente , rumiando . Pero cuando está conmigo , entonces no aparta los ojos de mi mujer y yo , con
Neigh: Detrás de él, las vacas dormitaban al sol ya caliente, rumiando.
Text: dormitaban al sol ya caliente , rumiando . Pero cuando está conmigo , entonces no aparta los ojos de mi mujer y yo , con
Neigh: Pero cuando está conmigo, entonces no aparta los ojos de ellos.
Text: dormitaban al sol ya caliente , rumiando . Pero cuando está conmigo , entonces no aparta los ojos de mi mujer y yo , con
Neigh: Y juro que fueron fuertes las dos horas que pasamos mi mujer y yo, con la luz prendida hasta que amaneció, ella acostada, yo sentado en la cama, vigilando sin cesar la arpillera flotante.
Text: dormitaban al sol ya caliente , rumiando . Pero en el patio y Alfonso la llamó en silencio . Pasábanse horas sin oir el angustioso
Neigh: Celia, mi tía mayor, que había concluído de dormir la siesta, cruzó el patio y Alfonso la llamó en si

,output_text,idx,neighbor
0,"dormitaban al sol ya caliente , rumiando . Per...",1,"Detrás de él, las vacas dormitaban al sol ya c..."
1,"dormitaban al sol ya caliente , rumiando . Per...",2,"Pero cuando está conmigo, entonces no aparta l..."
2,"dormitaban al sol ya caliente , rumiando . Per...",3,Y juro que fueron fuertes las dos horas que pa...
3,"dormitaban al sol ya caliente , rumiando . Per...",1,"Celia, mi tía mayor, que había concluído de do..."
4,"dormitaban al sol ya caliente , rumiando . Per...",2,"Detrás de él, las vacas dormitaban al sol ya c..."
...,...,...,...
58,", peones de obraje , volvían a Posadas en el _...",2,"El _Silex_ volvió a Posadas, llevando con él a..."
59,", peones de obraje , volvían a Posadas en el _...",3,"Podeley, cuya fiebre anterior había tenido hon..."
60,peones de obraje volvían a Posadas en el _Sile...,1,"Podeley, labrador de madera, tornaba a los nue..."
61,peones de obraje volvían a Posadas en el _Sile...,2,#LOS MENSÚ#\n\n\n\n\nCayetano Maidana y Esteba...


### Distancia de Edición vs. Embeddings

Con n-gramas y distancia de edición se obtienen oraciones similares en las palabras y su orden de aparición en la oración (esto es por como se implementa dicha distancia). Como el modelo de generación se sobreajusta al texto de entrenamiento, la distancia de edición suele recuperar una oración muy similar.

Por otro lado, con sentence-transformers obtenemos oraciones que significan lo mismo (o que en significado están muy cerca) pero que pueden diferir en largo, palabras y orden de las mismas.

## Segunda parte: Clasificación de textos por autor

En esta segunda parte del laboratorio vamos a usar los datos limpios de `quiroga` y todo lo practicado de manipulación de tiras para hacer experimentos de clasificación.


Con el fin de construir modelos que intenten identificar los autores de cada oración según su estilo de escritura, vamos a incorporar dos libros más, una [selección de obras](https://www.gutenberg.org/ebooks/53552) de Gustavo Adolfo Bécquer y [El Gaucho Martín Fierro](https://es.wikipedia.org/wiki/El_Gaucho_Mart%C3%ADn_Fierro) de José Hernández.

In [83]:
%%capture
!wget https://www.gutenberg.org/cache/epub/53552/pg53552.txt
with open("pg53552.txt","r") as f:
    becquer_raw = f.read()

!wget https://www.gutenberg.org/cache/epub/14765/pg14765.txt
with open("pg14765.txt","r") as f:
    martin_fierro_raw = f.read()

### 4️⃣ Limpieza básica del nuevo texto


Análogamente a lo hecho en el ejercicio 0️⃣, limpien `becquer_raw` para que en la variable `becquer` quede:


* solamente el texto que está entre "Junio de 1868." y "FIN\n"
* sin aquellos saltos de línea que tengan a su derecha una palabra que comience con letra minúscula
* y borrando todas las subtiras "[Ilustración]"
  

In [84]:
#En esta celda les dejamos esta asignación para que el notebook sea funcional.
#Implementen en esta celda la limpieza correspondiente.
#becquer = becquer_raw

# 1. Extracción del texto
print("Extrayendo texto...")
becquer_text_extract_pattern = r"Junio de 1868\.(.*)FIN\n"  # Expresion regular
match = re.search(becquer_text_extract_pattern, becquer_raw, re.DOTALL)
if match:
    becquer_text = match.group(1).strip()  # Remuevo espacios en blanco
    print(f"Texto original: {len(becquer_raw)} caracteres. \nTexto extraído: {len(becquer_text)} caracteres.")
else:
    print("Algo anda mal!")

# 2. Limpieza saltos de línea
print("\n\nLimpiando saltos de línea en sentencias...")
breakline_lowercase_pattern = r"\n([a-z])"
becquer = re.sub(breakline_lowercase_pattern, r" \1", becquer_text)  # Reemplazo \n por un ' '
print(f"Texto original: {len(becquer_text)} caracteres. \nTexto extraído: {len(becquer)} caracteres.")

# 3. Borro subturas ilustración
becquer = becquer.replace("[Ilustración]", " ")

print("\n\n")
print("Texto Limpio\n")
print(becquer[:1000])

Extrayendo texto...
Texto original: 573195 caracteres. 
Texto extraído: 527105 caracteres.


Limpiando saltos de línea en sentencias...
Texto original: 527105 caracteres. 
Texto extraído: 527105 caracteres.



Texto Limpio

 




LEYENDAS




 

MAESE PÉREZ EL ORGANISTA


En Sevilla, en el mismo atrio de Santa Inés, y mientras esperaba que comenzase la Misa del Gallo, oí esta tradición á una demandadera del convento.

Como era natural, después de oirla, aguardé impaciente que comenzara la ceremonia, ansioso de asistir á un prodigio.

Nada menos prodigioso, sin embargo, que el órgano de Santa Inés, ni nada más vulgar que los insulsos motetes que nos regaló su organista aquella noche.

Al salir de la Misa, no pude por menos de decirle á la demandadera con aire de burla:

--¿En qué consiste que el órgano de maese Pérez suena ahora tan mal?

--¡Toma!--me contestó la vieja,--en que ese no es el suyo.

--¿No es el suyo? ¿Pues qué ha sido de él?

--Se cayó á pedazos de puro viejo, hace una po

Hagan lo mismo con `martin_fierro_raw`, guardando en la variable `martin_fierro`:
* el texto que está entre "Buenos Aires, diciembre de 1872." y "End of Project"
* sin aquellos saltos de línea que tengan a su derecha una palabra que comience con letra minúscula
* y borrando todos los números de entre 1 y 3 cifas, e.g. 7, 12, 178   

⚠️ Aclaración: *El Gaucho Martín Fierro* está originalmente escrito como un poema, por lo que los saltos de línea no tienen el mismo fin que en la prosa. Sin embargo, eliminaremos esos saltos de línea seguidos de minúsculas como convención, a fin de facilitar el ejercicio y hacer más parecido el texto a los guardados en `quiroga` y `becquer`.

In [85]:
#En esta celda les dejamos esta asignación para que el notebook sea funcional.
#Implementen en esta celda la limpieza correspondiente.
#martin_fierro = martin_fierro_raw

# 1. Extracción del texto
print("Extrayendo texto...")
martin_fierro_text_extract_pattern = r"Buenos Aires, diciembre de 1872\.(.*)End of Project"  # Expresion regular
match = re.search(martin_fierro_text_extract_pattern, martin_fierro_raw, re.DOTALL)
if match:
    martin_fierro_text = match.group(1).strip()  # Remuevo espacios en blanco
    print(f"Texto original: {len(martin_fierro_raw)} caracteres. \nTexto extraído: {len(martin_fierro_text)} caracteres.")
else:
    print("Algo anda mal!")

# 2. Limpieza saltos de línea
print("\n\nLimpiando saltos de línea en sentencias...")
breakline_lowercase_pattern = r"\n([a-z])"
martin_fierro = re.sub(breakline_lowercase_pattern, r" \1", martin_fierro_text)  # Reemplazo \n por un ' '
print(f"Texto original: {len(martin_fierro_text)} caracteres. \nTexto extraído: {len(martin_fierro)} caracteres.")

# 3. Borro numeros de 1-3 cifras
martin_fierro = martin_fierro.replace("[Ilustración]", " ")
martin_fierro = re.sub(r'\b\d{1,3}\b', ' ', martin_fierro)

print("\n\n")
print("Texto Limpio\n")
print(martin_fierro[:1000])

Extrayendo texto...
Texto original: 85812 caracteres. 
Texto extraído: 62129 caracteres.


Limpiando saltos de línea en sentencias...
Texto original: 62129 caracteres. 
Texto extraído: 62129 caracteres.



Texto Limpio

El Gaucho Martín Fierro


I - Cantor y Gaucho.

 
Aquí me pongo a cantar
Al compás de la vigüela,
Que el hombre que lo desvela
Una pena estraordinaria
Como la ave solitaria
Con el cantar se consuela.

 
Pido a los Santos del Cielo
Que ayuden mi pensamiento;
Les pido en este momento
Que voy a cantar mi historia
Me refresquen la memoria
Y aclaren mi entendimiento.

 
Vengan Santos milagrosos,
Vengan todos en mi ayuda,
Que la lengua se me añuda
Y se me turba la vista;
Pido a Dios que me asista
En una ocasión tan ruda.

 
Yo he visto muchos cantores,
Con famas bien obtenidas,
Y que después de adquiridas
No las quieren sustentar
Parece que sin largar se cansaron en partidas

 
Mas ande otro criollo pasa
Martín Fierro ha de pasar; nada lo hace recular ni los fantasmas lo espa

### Creación del corpus de clasificación

En la siguiente celda, ya implementada, se crea el corpus y se subdivide en conjuntos de entrenamiento (`corpus_sentences_train`, `corpus_authors_train`), desarrollo (`corpus_sentences_dev`, `corpus_authors_dev`) y evaluación  (`corpus_sentences_test`, `corpus_authors_test`).

💡Cada conjunto está a su vez subdividido en `corpus_sentences` y `corpus_authors`, donde los textos que pertenecen a `quiroga` están señalizados por el `0`, los de `becquer` por el `1` y los de `martin_fierro` por el `2`. Esto facilita el algoritmo de partición y también la evaluación.

In [86]:
from sklearn.model_selection import train_test_split

quiroga_sentences = [x for x in nltk.sent_tokenize(quiroga,  language='spanish') if len(x)>4]
corpus_sentences = quiroga_sentences
corpus_authors = [0]*len(quiroga_sentences)

becquer_sentences = [x for x in nltk.sent_tokenize(becquer,  language='spanish') if len(x)>4]
corpus_sentences += becquer_sentences
corpus_authors += [1]*len(becquer_sentences)

martin_fierro_sentences = [x for x in nltk.sent_tokenize(martin_fierro,  language='spanish') if len(x)>4]
corpus_sentences += martin_fierro_sentences
corpus_authors += [2]*len(martin_fierro_sentences)

corpus_sentences_train, corpus_sentences_other, corpus_authors_train, corpus_authors_other = train_test_split(
    corpus_sentences, corpus_authors,
    test_size=0.3, train_size=0.7,
    random_state=2024, shuffle=True, stratify=corpus_authors)

corpus_sentences_dev, corpus_sentences_test, corpus_authors_dev, corpus_authors_test = train_test_split(
    corpus_sentences_other, corpus_authors_other,
    test_size=0.5, train_size=0.5,
    random_state=2024, shuffle=True, stratify=corpus_authors_other)

### 5️⃣ Análisis del corpus

Para entender mejor cómo está compuesto el corpus, hallen:
- la cantidad de instancias en los subconjuntos `train`, `dev` y `test`
- la cantidad de instancias de cada una de las clases (*quiroga*, *becquer*  y *martin_fierro*) en cada uno de los subconjuntos

In [87]:
from collections import Counter

#Completar

# Cantidad de instancias en cada corpus
print("---- TRAIN ----")
print(f"sentencias={len(corpus_sentences_train)}")
counter_dict = Counter(corpus_authors_train)
print(f"quiroga={counter_dict.get(0, 0)} becquer={counter_dict.get(1, 0)} martin_fierro={counter_dict.get(2, 0)}")

print("\n---- DEV ----")
print(len(corpus_sentences_dev))
counter_dict = Counter(corpus_authors_dev)
print(f"quiroga={counter_dict.get(0, 0)} becquer={counter_dict.get(1, 0)} martin_fierro={counter_dict.get(2, 0)}")

print("\n---- TEST ----")
print(len(corpus_sentences_test))
counter_dict = Counter(corpus_authors_test)
print(f"quiroga={counter_dict.get(0, 0)} becquer={counter_dict.get(1, 0)} martin_fierro={counter_dict.get(2, 0)}")


---- TRAIN ----
sentencias=4530
quiroga=2469 becquer=1727 martin_fierro=334

---- DEV ----
971
quiroga=529 becquer=370 martin_fierro=72

---- TEST ----
971
quiroga=530 becquer=370 martin_fierro=71


### Evaluación del clasificador aleatorio (línea base)

Ahora que tenemos los conjuntos definidos, ya podemos entrenar modelos. Pero antes, siempre es bueno ver cómo se comportaría un algoritmo sencillo que consista en una heurística con pocas reglas o en una clasificación aleatoria. A este modelo sencillo, que sirve como punto de comparación, le solemos llamar *línea base* (baseline).

En la siguiente celda les mostramos la evaluación sobre `dev` de la predicción aleatoria, para que tengan una línea base con la que comparar los modelos que construirán a continuación.

In [88]:
import random
from sklearn.metrics import f1_score, classification_report

random_prediction = [random.choice([0,1,2]) for i in range(len(corpus_authors_dev))]
print("F-Score macro: " + str(round(f1_score(corpus_authors_dev, random_prediction, average='macro')*100, 2))) # Se imprime la medida F
print(classification_report(corpus_authors_dev, random_prediction, target_names=['quiroga','becquer','martin_fierro'])) # Se evalúa sobre dev

F-Score macro: 30.17
               precision    recall  f1-score   support

      quiroga       0.57      0.35      0.43       529
      becquer       0.38      0.33      0.35       370
martin_fierro       0.07      0.33      0.12        72

     accuracy                           0.34       971
    macro avg       0.34      0.34      0.30       971
 weighted avg       0.46      0.34      0.38       971



### 6️⃣ Entrenamiento y evaluación de modelos en el conjunto de desarrollo (dev)

Ahora que vimos la medida F1 en `dev` para el clasificador aleatorio, ya pueden entrenar modelos y evaluarlos intentando superar esa línea base. En la siguiente celda mostramos el entrenamiento sobre `train` de un modelo sencillo y su posterior evaluación sobre `dev`, donde usamos [bag of words](https://scikit-learn.org/stable/modules/feature_extraction.html#the-bag-of-words-representation) para representar numéricamente los textos y [Support Vector Machines](https://scikit-learn.org/stable/modules/svm.html) como algoritmo de clasificación.

Entrenen con `train` y evalúen sobre `dev` al menos 4 modelos, explorando varios algoritmos de clasificación y, si quieren, también preprocesando los textos usando  pipelines de preprocesamiento con pasos complementarios a los ya desarrollados en 4️⃣.

💡Les recomendamos que comiencen probando vectorizar con [bag of words](https://scikit-learn.org/stable/modules/feature_extraction.html#the-bag-of-words-representation) o [TF-IDF](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html), y clasificando con cualquier algoritmo disponible en [sklearn](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html). Sin embargo, vemos positivo que exploren **CUALQUIER** enfoque: pueden probar usando grandes modelos de lenguaje como LLAMA o Mistral, o también vectorizando con vectores contextuales como los *sentence transformers* usados en el ejercicio 3️⃣. En [HuggingFace](https://huggingface.co/) pueden encontrar muchísimos modelos para probar.

💡Tengan en cuenta que si usan un pipeline de preprocesamiento para procesar los textos al entrenar, también tendrán que usarlo para transformar los textos al predecir.

A modo de referencia la siguiente tabla enumer alos experimentos que nos proponemos realizar de aquí en adelante:

|vectors | model |
| ----	 | ------|
| BoW    | SVM   |
| TF-IDF | SVM   |
|BoW     |MLPClassifier|
|TF-IDF  |MLPClassifier|
|TF-IDF + SMOTE |SVM|
|TF-IDF + SMOTE |MLPClassifier|
|SentenceTransformers |SVM|
|SentenceTransformers |MLPClassifier|


### 1. Bow + SVM

El siguiente modelo combina Bag-of-Words con un modelo SVM.

In [89]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC


bow_vectorizer = CountVectorizer(ngram_range=(2,4), strip_accents= 'unicode') # n-gramas a nivel de palabra
clf = SVC() # Prueben acá con varios modelos

training_features = bow_vectorizer.fit_transform(corpus_sentences_train) # Se vectorizan los textos de train
clf.fit(training_features, corpus_authors_train) # Se entrena el clasificador usando los textos vectorizados

dev_features = bow_vectorizer.transform(corpus_sentences_dev) # Se vectorizan los textos de dev
prediction = clf.predict(dev_features) # Se predicen los autores de cada texto (ya vectorizado en la línea anterior)

print("F-Score macro: " + str(round(f1_score(corpus_authors_dev, prediction, average='macro')*100, 2)))
print(classification_report(corpus_authors_dev, prediction, target_names=['quiroga','becquer','martin_fierro']))

F-Score macro: 33.49
               precision    recall  f1-score   support

      quiroga       0.58      1.00      0.73       529
      becquer       1.00      0.16      0.27       370
martin_fierro       0.00      0.00      0.00        72

     accuracy                           0.60       971
    macro avg       0.53      0.39      0.33       971
 weighted avg       0.70      0.60      0.50       971



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Observaciones**
* El modelo se sobre ajusta a los textos de quiroga con un alto recall a costa de una baja precision. Esto es esperable de un modelo sencillo, con un dataset tan desbalanceado.
* No se reconoce ningún ejemplo de la clase martin_fierro. Esto es esperable también ya que se tienen muy pocos ejemplos (72).
* En resumen el modelo no es bueno

### 2. TF-IDF + SVM

El siguiente modelo combina TF-IDF, que es una estrategia de vectorización complementaria a BoW que permite obtener mejores resultados, sobre todo al considerar palabras menos frecuentes, con un modelo SVM.

In [90]:
from sklearn.feature_extraction.text import TfidfTransformer

# Setup
bow_vectorizer = CountVectorizer(ngram_range=(2,4), strip_accents= 'unicode') # n-gramas a nivel de palabra
tf_idf = TfidfTransformer(use_idf=True)
clf = SVC() # Prueben acá con varios modelos

# 1. Fit Modelo
training_features = bow_vectorizer.fit_transform(corpus_sentences_train) # Se vectorizan los textos de train con BoW
training_features = tf_idf.fit_transform(training_features)  # Se mejoran con TF-IDF
clf.fit(training_features, corpus_authors_train) # Se entrena el clasificador usando los textos vectorizados

# 2. Eval Modelo
dev_features = bow_vectorizer.transform(corpus_sentences_dev) # Se vectorizan los textos de dev con BoW
dev_features = tf_idf.fit_transform(dev_features)  # Se mejoran con TF-IDF
prediction = clf.predict(dev_features) # Se predicen los autores de cada texto (ya vectorizado en la línea anterior)

print("F-Score macro: " + str(round(f1_score(corpus_authors_dev, prediction, average='macro')*100, 2)))
print(classification_report(corpus_authors_dev, prediction, target_names=['quiroga','becquer','martin_fierro']))

F-Score macro: 47.7
               precision    recall  f1-score   support

      quiroga       0.69      0.90      0.78       529
      becquer       0.75      0.58      0.65       370
martin_fierro       0.00      0.00      0.00        72

     accuracy                           0.71       971
    macro avg       0.48      0.49      0.48       971
 weighted avg       0.66      0.71      0.67       971



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Observaciones**
* Con TF-IDF se obtienen mejores resultados, en particular tanto la precision como el recall se encuentran más balanceados entre las clases dominantes (quiroga y becquer).
* Persiste la no identificación de los textos de martin_fierro

### 3. BoW + MLP Classifier

El siguiente modelo combina BoW, con un modelo de redes neuronales, capaz de sobre ajustarse mejor a los datos de entrenamiento.

In [91]:
from sklearn.neural_network import MLPClassifier

# Setup
bow_vectorizer = CountVectorizer(ngram_range=(2,4), strip_accents= 'unicode') # n-gramas a nivel de palabra
clf = MLPClassifier(hidden_layer_sizes=(50,),
                    activation='relu',
                    solver='adam',
                    alpha=0.0001,
                    batch_size='auto',
                    learning_rate='constant',
                    learning_rate_init=0.001,
                    max_iter=50,
                    shuffle=True,
                    random_state=1,
                    verbose=True,
                    momentum=0.9,
                    nesterovs_momentum=True,
                    early_stopping=True,
                    validation_fraction=0.1)

# 1. Fit Modelo
training_features = bow_vectorizer.fit_transform(corpus_sentences_train) # Se vectorizan los textos de train con BoW
clf.fit(training_features, corpus_authors_train) # Se entrena el clasificador usando los textos vectorizados

# 2. Eval Modelo
dev_features = bow_vectorizer.transform(corpus_sentences_dev) # Se vectorizan los textos de dev con BoW
dev_features = tf_idf.fit_transform(dev_features)  # Se mejoran con TF-IDF
prediction = clf.predict(dev_features) # Se predicen los autores de cada texto (ya vectorizado en la línea anterior)

print("F-Score macro: " + str(round(f1_score(corpus_authors_dev, prediction, average='macro')*100, 2)))
print(classification_report(corpus_authors_dev, prediction, target_names=['quiroga','becquer','martin_fierro']))

Iteration 1, loss = 1.12994493
Validation score: 0.536424
Iteration 2, loss = 0.55404840
Validation score: 0.818985
Iteration 3, loss = 0.29159150
Validation score: 0.801325
Iteration 4, loss = 0.18706852
Validation score: 0.792494
Iteration 5, loss = 0.13609585
Validation score: 0.785872
Iteration 6, loss = 0.10727612
Validation score: 0.779249
Iteration 7, loss = 0.08867490
Validation score: 0.777042
Iteration 8, loss = 0.07604580
Validation score: 0.777042
Iteration 9, loss = 0.06683892
Validation score: 0.777042
Iteration 10, loss = 0.05995841
Validation score: 0.777042
Iteration 11, loss = 0.05452054
Validation score: 0.772627
Iteration 12, loss = 0.05022459
Validation score: 0.768212
Iteration 13, loss = 0.04675281
Validation score: 0.770419
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
F-Score macro: 46.14
               precision    recall  f1-score   support

      quiroga       0.61      0.99      0.76       529
      becquer    

**Observaciones**
* El modelo demuestra mayor capacidad para sobre ajustar los datos, al punto que logra reconocer algunas sentencias de martin_fierro (muy poquitas).
* Por otro se evidencia todavía un sesgo a predecir "quiroga" evidentemente por el desbalance de clases.
* Si bien la red neuronal parece tener mayor capacidad de modelado que SVM no es una buena alternativa.
* Probemos MLP con TF-IDF

### 4. TF-IDF + MLP Classifier

El siguiente modelo combina TF-IDF (ya vimos que permite obtener mejores resultados), con un modelo de redes neuronales.

In [92]:
# Setup
bow_vectorizer = CountVectorizer(ngram_range=(2,4), strip_accents= 'unicode') # n-gramas a nivel de palabra
tf_idf = TfidfTransformer(use_idf=True)
clf = MLPClassifier(hidden_layer_sizes=(50,),
                    activation='relu',
                    solver='adam',
                    alpha=0.0001,
                    batch_size='auto',
                    learning_rate='constant',
                    learning_rate_init=0.001,
                    max_iter=50,
                    shuffle=True,
                    random_state=1,
                    verbose=True,
                    momentum=0.9,
                    nesterovs_momentum=True,
                    early_stopping=True,
                    validation_fraction=0.1)

# 1. Fit Modelo
training_features = bow_vectorizer.fit_transform(corpus_sentences_train) # Se vectorizan los textos de train con BoW
training_features = tf_idf.fit_transform(training_features)  # Se mejoran con TF-IDF
clf.fit(training_features, corpus_authors_train) # Se entrena el clasificador usando los textos vectorizados

# 2. Eval Modelo
dev_features = bow_vectorizer.transform(corpus_sentences_dev) # Se vectorizan los textos de dev con BoW
dev_features = tf_idf.fit_transform(dev_features)  # Se mejoran con TF-IDF
prediction = clf.predict(dev_features) # Se predicen los autores de cada texto (ya vectorizado en la línea anterior)

print("F-Score macro: " + str(round(f1_score(corpus_authors_dev, prediction, average='macro')*100, 2)))
print(classification_report(corpus_authors_dev, prediction, target_names=['quiroga','becquer','martin_fierro']))

Iteration 1, loss = 1.19801521
Validation score: 0.072848
Iteration 2, loss = 1.00157398
Validation score: 0.576159
Iteration 3, loss = 0.77302136
Validation score: 0.580574
Iteration 4, loss = 0.56690765
Validation score: 0.618102
Iteration 5, loss = 0.39883285
Validation score: 0.673289
Iteration 6, loss = 0.28434121
Validation score: 0.682119
Iteration 7, loss = 0.20937706
Validation score: 0.693157
Iteration 8, loss = 0.15936651
Validation score: 0.697572
Iteration 9, loss = 0.12548816
Validation score: 0.701987
Iteration 10, loss = 0.10222102
Validation score: 0.706402
Iteration 11, loss = 0.08592488
Validation score: 0.706402
Iteration 12, loss = 0.07407344
Validation score: 0.704194
Iteration 13, loss = 0.06519518
Validation score: 0.704194
Iteration 14, loss = 0.05838758
Validation score: 0.706402
Iteration 15, loss = 0.05317802
Validation score: 0.710817
Iteration 16, loss = 0.04899312
Validation score: 0.715232
Iteration 17, loss = 0.04565946
Validation score: 0.715232
Iterat

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


**Observaciones**
* Resultados mucho mejores y balanceados para Quiroga y Becquer
* Aun no resolvemos el problema con martin_fierro

### 5. TF-IDF + SMOTE + MLP Classifier

El principal problema que tienen hasta ahora los modelos entrenados, es que no son capaces de modelar con cierto grado de equidad las diferentes clases. En particular son muy buenos prediciendo textos de la clase "Quiroga", mediocres prediciendo textos de la clase "Becquer" y muy malos prediciendo los textos de la clase "Martin Fierro". Esto se puede deber a que el corpus se encuentra desbalanceado, con muy pocos ejemplos de estas dos últimas clases.

Una forma de solventar este problema, es mediante técnicas de undersampling/oversampling, para balancear el dataset. En el siguiente experimento utilizamos SMOTE de `imblearn` para entrenar un modelo TF-IDF + MLP Classifier sobre un dataset más balanceado.


In [93]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neural_network import MLPClassifier
import numpy as np
from imblearn.over_sampling import SMOTE

def featurize_model_5(corpus_sentences):
  features = bow_vectorizer.fit_transform(corpus_sentences) # Se vectorizan los textos de train con BoW
  features = tf_idf.fit_transform(features)  # Se mejoran con TF-IDF
  return features

def train_model_5(corpus_sentences_train, corpus_authors_train):

  # Setup
  bow_vectorizer = CountVectorizer(ngram_range=(2,4), strip_accents= 'unicode') # n-gramas a nivel de palabra
  tf_idf = TfidfTransformer(use_idf=True)

  clf = MLPClassifier(hidden_layer_sizes=(50,),
                      activation='relu',
                      solver='adam',
                      alpha=0.0001,
                      batch_size='auto',
                      learning_rate='constant',
                      learning_rate_init=0.001,
                      max_iter=100,
                      shuffle=True,
                      random_state=1,
                      verbose=True,
                      momentum=0.9,
                      nesterovs_momentum=True,
                      early_stopping=True,
                      validation_fraction=0.1)

  # 1. Fit Modelo
  training_features = bow_vectorizer.fit_transform(corpus_sentences_train) # Se vectorizan los textos de train con BoW
  training_features = tf_idf.fit_transform(training_features)  # Se mejoran con TF-IDF

  # 2. Apply SMOTE to the training set
  smote = SMOTE(random_state=42)
  training_features_resampled, corpus_authors_train_resampled = smote.fit_resample(training_features, corpus_authors_train)

  clf.fit(training_features_resampled, corpus_authors_train_resampled) # Se entrena el clasificador usando los textos vectorizados

  return clf

# Train model 5
model_5 = train_model_5(corpus_sentences_train, corpus_authors_train)

# Eval model 5
dev_features = bow_vectorizer.transform(corpus_sentences_dev) # Se vectorizan los textos de dev con BoW
dev_features = tf_idf.fit_transform(dev_features)  # Se mejoran con TF-IDF
prediction = model_5.predict(dev_features) # Se predicen los autores de cada texto (ya vectorizado en la línea anterior)

print("F-Score macro: " + str(round(f1_score(corpus_authors_dev, prediction, average='macro')*100, 2)))
print(classification_report(corpus_authors_dev, prediction, target_names=['quiroga','becquer','martin_fierro']))

Iteration 1, loss = 1.06232207
Validation score: 0.423752
Iteration 2, loss = 0.79477710
Validation score: 0.796221
Iteration 3, loss = 0.51857686
Validation score: 0.851552
Iteration 4, loss = 0.33732659
Validation score: 0.866397
Iteration 5, loss = 0.23591078
Validation score: 0.887989
Iteration 6, loss = 0.17978490
Validation score: 0.894737
Iteration 7, loss = 0.14581278
Validation score: 0.904184
Iteration 8, loss = 0.12342339
Validation score: 0.910931
Iteration 9, loss = 0.10762175
Validation score: 0.914980
Iteration 10, loss = 0.09600583
Validation score: 0.909582
Iteration 11, loss = 0.08711447
Validation score: 0.910931
Iteration 12, loss = 0.08019881
Validation score: 0.904184
Iteration 13, loss = 0.07468790
Validation score: 0.900135
Iteration 14, loss = 0.07017213
Validation score: 0.860999
Iteration 15, loss = 0.06650776
Validation score: 0.851552
Iteration 16, loss = 0.06342872
Validation score: 0.850202
Iteration 17, loss = 0.06082376
Validation score: 0.850202
Iterat

**Observaciones**
* Hermoso

### 6. Sentence Transformers + SMOTE + SVM

In [94]:
from sentence_transformers import SentenceTransformer

def featurize_model_6(corpus_sentences, model_vectorize):
  features = model_vectorize.encode(corpus_sentences, normalize_embeddings=True)
  return features

def train_model_6(corpus_sentences_train, corpus_authors_train, model_vectorize):
  clf = SVC() # Prueben acá con varios modelos

  # 1. Fit Modelo
  training_features = featurize_model_6(corpus_sentences_train, model_vectorize)

  # 2. Apply SMOTE to the training set
  smote = SMOTE(random_state=42)
  training_features_resampled, corpus_authors_train_resampled = smote.fit_resample(training_features, corpus_authors_train)

  # 3. Fit model
  clf.fit(training_features_resampled, corpus_authors_train_resampled) # Se entrena el clasificador usando los textos vectorizados

  return clf

# Para vectorize
model_vectorize = SentenceTransformer('intfloat/multilingual-e5-large')
# Train model 6
model_6 = train_model_6(corpus_sentences_train, corpus_authors_train, model_vectorize)

# 3. Eval Modelo
dev_features = featurize_model_6(corpus_sentences_dev, model_vectorize)
prediction = model_6.predict(dev_features) # Se predicen los autores de cada texto (ya vectorizado en la línea anterior)

print("F-Score macro: " + str(round(f1_score(corpus_authors_dev, prediction, average='macro')*100, 2)))
print(classification_report(corpus_authors_dev, prediction, target_names=['quiroga','becquer','martin_fierro']))

F-Score macro: 85.48
               precision    recall  f1-score   support

      quiroga       0.86      0.93      0.89       529
      becquer       0.90      0.80      0.85       370
martin_fierro       0.83      0.82      0.83        72

     accuracy                           0.87       971
    macro avg       0.86      0.85      0.85       971
 weighted avg       0.87      0.87      0.87       971



**OBSERVACIONES**

In [95]:
from sentence_transformers import SentenceTransformer

def featurize_model_7(corpus_sentences, model_vectorize):
  features = model_vectorize.encode(corpus_sentences, normalize_embeddings=True)
  return features

def train_model_7(corpus_sentences_train, corpus_authors_train, model_vectorize):
  clf = MLPClassifier(hidden_layer_sizes=(50,),
                      activation='relu',
                      solver='adam',
                      alpha=0.0001,
                      batch_size='auto',
                      learning_rate='constant',
                      learning_rate_init=0.001,
                      max_iter=50,
                      shuffle=True,
                      random_state=1,
                      verbose=True,
                      momentum=0.9,
                      nesterovs_momentum=True,
                      early_stopping=True,
                      validation_fraction=0.1)

  # 1. Fit Modelo
  training_features = model.encode(corpus_sentences_train, normalize_embeddings=True)

  # 2. Apply SMOTE to the training set
  smote = SMOTE(random_state=42)
  training_features_resampled, corpus_authors_train_resampled = smote.fit_resample(training_features, corpus_authors_train)

  clf.fit(training_features_resampled, corpus_authors_train_resampled) # Se entrena el clasificador usando los textos vectorizados

  return clf

# Para vectorize
model_vectorize = SentenceTransformer('intfloat/multilingual-e5-large')

# Train model 7
model_7 = train_model_7(corpus_sentences_train, corpus_authors_train, model_vectorize)

# 3. Eval Modelo
dev_features = featurize_model_6(corpus_sentences_dev, model_vectorize)
prediction = model_7.predict(dev_features) # Se predicen los autores de cada texto (ya vectorizado en la línea anterior)

print("F-Score macro: " + str(round(f1_score(corpus_authors_dev, prediction, average='macro')*100, 2)))
print(classification_report(corpus_authors_dev, prediction, target_names=['quiroga','becquer','martin_fierro']))

Iteration 1, loss = 1.07346558
Validation score: 0.705803
Iteration 2, loss = 0.98381915
Validation score: 0.790823
Iteration 3, loss = 0.87534139
Validation score: 0.800270
Iteration 4, loss = 0.76244772
Validation score: 0.801619
Iteration 5, loss = 0.66241874
Validation score: 0.807018
Iteration 6, loss = 0.58138941
Validation score: 0.839406
Iteration 7, loss = 0.52062089
Validation score: 0.848853
Iteration 8, loss = 0.47123973
Validation score: 0.859649
Iteration 9, loss = 0.43545546
Validation score: 0.854251
Iteration 10, loss = 0.40727798
Validation score: 0.870445
Iteration 11, loss = 0.38169067
Validation score: 0.866397
Iteration 12, loss = 0.36258079
Validation score: 0.875843
Iteration 13, loss = 0.34320657
Validation score: 0.885290
Iteration 14, loss = 0.32929227
Validation score: 0.892038
Iteration 15, loss = 0.31589525
Validation score: 0.901484
Iteration 16, loss = 0.30321333
Validation score: 0.905533
Iteration 17, loss = 0.29305392
Validation score: 0.906883
Iterat

**OBSERVACIONES**

In [96]:
# from sklearn.utils.class_weight import compute_class_weight
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.neural_network import MLPClassifier
# import numpy as np
# from imblearn.over_sampling import SMOTE

# # Setup
# bow_vectorizer = CountVectorizer(ngram_range=(2,4), strip_accents= 'unicode') # n-gramas a nivel de palabra
# tf_idf = TfidfTransformer(use_idf=True)
# clf = SVC() # Prueben acá con varios modelos

# # 1. Fit Modelo
# training_features = bow_vectorizer.fit_transform(corpus_sentences_train) # Se vectorizan los textos de train con BoW
# training_features = tf_idf.fit_transform(training_features)  # Se mejoran con TF-IDF

# # 2. Apply SMOTE to the training set
# smote = SMOTE(random_state=42)
# training_features_resampled, corpus_authors_train_resampled = smote.fit_resample(training_features, corpus_authors_train)
# clf.fit(training_features_resampled, corpus_authors_train_resampled) # Se entrena el clasificador usando los textos vectorizados

# # 3. Eval Modelo
# dev_features = bow_vectorizer.transform(corpus_sentences_dev) # Se vectorizan los textos de dev con BoW
# dev_features = tf_idf.fit_transform(dev_features)  # Se mejoran con TF-IDF
# prediction = clf.predict(dev_features) # Se predicen los autores de cada texto (ya vectorizado en la línea anterior)

# print("F-Score macro: " + str(round(f1_score(corpus_authors_dev, prediction, average='macro')*100, 2)))
# print(classification_report(corpus_authors_dev, prediction, target_names=['quiroga','becquer','martin_fierro']))

### 7️⃣Evaluación final de modelos en el conjunto de evaluación (test)

Como última actividad del laboratorio, elijan los tres modelos que mejores predicciones hayan logrado sobre `dev` y evalúenlos sobre `test`.

### 1. TF-IDF + SMOTE + MLP Classifier

In [97]:
# Eval Modelo
test_features = bow_vectorizer.transform(corpus_sentences_test) # Se vectorizan los textos de dev con BoW
test_features = tf_idf.fit_transform(test_features)  # Se mejoran con TF-IDF
prediction = model_5.predict(test_features) # Se predicen los autores de cada texto (ya vectorizado en la línea anterior)

print("F-Score macro: " + str(round(f1_score(corpus_authors_test, prediction, average='macro')*100, 2)))
print(classification_report(corpus_authors_test, prediction, target_names=['quiroga','becquer','martin_fierro']))

F-Score macro: 76.61
               precision    recall  f1-score   support

      quiroga       0.86      0.85      0.86       530
      becquer       0.78      0.84      0.81       370
martin_fierro       0.75      0.55      0.63        71

     accuracy                           0.82       971
    macro avg       0.80      0.75      0.77       971
 weighted avg       0.82      0.82      0.82       971



### 2. Sentence Transformers + SMOTE + SVM



In [98]:
# Eval Modelo
test_features = featurize_model_6(corpus_sentences_test, model_vectorize)
prediction = model_6.predict(test_features) # Se predicen los autores de cada texto (ya vectorizado en la línea anterior)

print("F-Score macro: " + str(round(f1_score(corpus_authors_test, prediction, average='macro')*100, 2)))
print(classification_report(corpus_authors_test, prediction, target_names=['quiroga','becquer','martin_fierro']))

F-Score macro: 86.6
               precision    recall  f1-score   support

      quiroga       0.86      0.93      0.89       530
      becquer       0.88      0.77      0.82       370
martin_fierro       0.86      0.90      0.88        71

     accuracy                           0.87       971
    macro avg       0.87      0.87      0.87       971
 weighted avg       0.87      0.87      0.87       971



### 3. Sentence Transformers + SMOTE + MLP Classifier



In [99]:
# Eval Modelo
test_features = featurize_model_7(corpus_sentences_test, model_vectorize)
prediction = model_7.predict(test_features) # Se predicen los autores de cada texto (ya vectorizado en la línea anterior)

print("F-Score macro: " + str(round(f1_score(corpus_authors_test, prediction, average='macro')*100, 2)))
print(classification_report(corpus_authors_test, prediction, target_names=['quiroga','becquer','martin_fierro']))

F-Score macro: 83.73
               precision    recall  f1-score   support

      quiroga       0.86      0.89      0.88       530
      becquer       0.85      0.77      0.81       370
martin_fierro       0.74      0.93      0.82        71

     accuracy                           0.85       971
    macro avg       0.82      0.86      0.84       971
 weighted avg       0.85      0.85      0.85       971

